# Basic Setup

In [1]:
## Import libraries and basic setups
import glob
import os
import sys
import xml.etree.ElementTree as ET
import json
import time
import pytz
import datetime as dt

## util libs
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
from collections import defaultdict
import ast

##iport sklearn and time 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, recall_score, precision_score, f1_score,
    roc_auc_score
)
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import load_arrow_head
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.utils.class_weight import compute_class_weight

from sklearn.linear_model import LinearRegression

#Import function to calculate time run
import time

def your_function():
    time.sleep(1)

#Import function to calculate Initial_EF and Pre_pulse_EF 
def Index_EF(x):
    # Ensure 'Time' is datetime
    x['Time'] = pd.to_datetime(x['Time']).dt.floor('D')

    # Group by the 'Time' column and calculate the mean for each day
    daily_mean = x.groupby('Time')['Daily_EF'].mean().reset_index()

    # Set 'Time' as the DataFrame index again for rolling calculation
    daily_mean.set_index('Time', inplace=True)

    # Calculate the rolling mean of the past 14 days, excluding the current day
    daily_mean['Pre_pulse_EF'] = daily_mean['Daily_EF'].rolling(window=15, min_periods=10).mean().shift(2)


    # Calculate the rolling mean of 2 on the shifted data, then shift upwards by 1 to align
    daily_mean['Before_EF'] = daily_mean['Daily_EF'].rolling(window=2, min_periods=1).mean().shift(1)
    daily_mean['After_EF'] = daily_mean['Daily_EF'].rolling(window=2, min_periods = 1).mean().shift(-1)

    daily_mean['Initial_EF'] = daily_mean['After_EF'] - daily_mean['Before_EF']

    #merge 2 df
    x_final = pd.merge(x, daily_mean[['Initial_EF', 'Pre_pulse_EF', 'After_EF']], on='Time', how='outer')
    #drop na value again
    x_final = x_final.dropna()
    
    return (x_final)



def change_NEE(x):
    # Ensure 'Time' is datetime
    x['Time'] = pd.to_datetime(x['Time']).dt.floor('D')

    # Group by the 'Time' column and calculate the mean for each day
    daily_mean = x.groupby('Time')['NEE_VUT_REF'].mean().reset_index()

    # Set 'Time' as the DataFrame index again for rolling calculation
    daily_mean.set_index('Time', inplace=True)
    # Calculate the rolling mean of the past 14 days, excluding the current day
    daily_mean['Pre_pulse_NEE'] = daily_mean['NEE_VUT_REF'].rolling(window=3, min_periods=1).mean().shift(2)

    # Calculate the rolling mean of 2 on the shifted data, then shift upwards by 1 to align
    daily_mean['Before_NEE'] = daily_mean['NEE_VUT_REF'].rolling(window=2, min_periods=1).mean().shift(1)
    daily_mean['After_NEE'] = daily_mean['NEE_VUT_REF'].rolling(window=2, min_periods = 1).mean().shift(-1)

    daily_mean['Initial_NEE'] = daily_mean['After_NEE'] - daily_mean['Before_NEE']
    daily_mean['Mean_NEE'] = daily_mean['NEE_VUT_REF']

    #merge 2 df
    x_final = pd.merge(x, daily_mean[['Initial_NEE', 'Pre_pulse_NEE', 'Mean_NEE', "After_NEE"]], on='Time', how='outer')
    #drop na value again
    x_final = x_final.dropna()
    
    return (x_final)


from joblib import dump, load

def create_sop_eop_df(df):
    # Ensure 'Time' column is in datetime format and sort by 'Time'
    #df['Time'] = pd.to_datetime(df['Time'])
    #df.sort_values('Time', inplace=True)

    # Detect changes in pulse status
    df['shifted'] = df['pulse_pred'].shift(1)
    df['next_shifted'] = df['pulse_pred'].shift(-1)
    
    # Identify changes where a pulse starts or ends
    df['change_start'] = (df['pulse_pred'] == 1) & (df['shifted'] != 1)
    df['change_end'] = (df['pulse_pred'] == 1) & (df['next_shifted'] != 1)

    # Start of Pulse (SOP): Where pulse starts
    df['SOP'] = df['Time'].where(df['change_start'])

    # End of Pulse (EOP): Where pulse ends
    df['EOP'] = df['Time'].where(df['change_end'])

    # Compile SOP and EOP into a new DataFrame
    sop_eop_df = pd.DataFrame({
        'SOP': df['SOP'].dropna().reset_index(drop=True),
        'EOP': df['EOP'].dropna().reset_index(drop=True)
    })

    # Remove rows where SOP equals EOP
    sop_eop_df = sop_eop_df[sop_eop_df['SOP'] != sop_eop_df['EOP']]
    return sop_eop_df

def remove_bad_pulse(group):
    threshold = 0.5
    total_count = group.shape[0]
    pulse_count = group['pulse_pred'].sum()
    pulse_day = 1 if (pulse_count / total_count) > threshold else 0
    group['pulse_pred'] = pulse_day
    return group


/Users/ngocnguyen/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/ngocnguyen/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
# Change to the specified directory
os.chdir("/Users/ngocnguyen/Library/CloudStorage/Box-Box/Projects/Birch_effect_paper/Results/Datasets/DT_interpolated_datasets_all_pulse/")


# Running random forest using multiple sites

In [3]:
combined_df = pd.read_csv('/Users/ngocnguyen/Library/CloudStorage/Box-Box/Projects/Birch_effect_paper/Results/Datasets/Random_forest/combined_time_series_data.csv')

In [4]:
#site with fewer than 5 years
site_few = ['AU-Dry', 'AU-Gin', 'AU-Rig', 'US-FR2', 'US-xNG']
#sites with few birch effects (< 20)
site_few_BE = ['AU-Dry', 'US-xNG', 'AU-Gin', 'US-FR2', 'AU-DaS']

In [6]:
#Filter for original data points
combined_df_sub = combined_df[(combined_df["NEE_VUT_REF_QC"] == 0) & (~combined_df['Location'].isin(site_few)) & (~combined_df['Location'].isin(site_few_BE))]

#Subset df for useful features and drop all NAs
features = ["Time_full", "Location", "Year", "Month", "Time", 'RECO_NT_VUT_REF', "GPP_NT_VUT_REF", "NEE_VUT_REF", "Daily_EF", "P_ERA", "pulse_cluster"]
combined_df_sub_fil = combined_df_sub[features].dropna()

In [92]:
#Calculate rewetting intensity as change_EF
df_final = combined_df_sub_fil.groupby('Location').apply(Index_EF)

/var/folders/x2/jr4h0w9926bbmtl5ysq4v78h0000gs/T/ipykernel_98331/3496692432.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = combined_df_sub_fil.groupby('Location').apply(Index_EF)


In [93]:
#calculate pulse intensity as change_NEE
df_final_NEE = combined_df_sub_fil.groupby('Location').apply(change_NEE)

/var/folders/x2/jr4h0w9926bbmtl5ysq4v78h0000gs/T/ipykernel_98331/331365674.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final_NEE = combined_df_sub_fil.groupby('Location').apply(change_NEE)


In [94]:
#add pulse intensity and rewetting intensity into the final csv
df_final['Initial_NEE'] = df_final_NEE['Initial_NEE'] 
df_final['Pre_pulse_NEE'] = df_final_NEE['Pre_pulse_NEE'] 
df_final['Mean_NEE'] = df_final_NEE['Mean_NEE'] 

In [95]:
#rename index column 
df_final.index.name = "Location_index"
df_final.reset_index(drop=True, inplace=True)

In [96]:
df_encoded = df_final.drop(['Time_full', 'GPP_NT_VUT_REF'], axis = 1)

In [97]:
#subset the dataset 
df_final_sub = df_encoded
# Identify the latest year for each location
latest_year_per_location = df_final_sub.groupby('Location')['Year'].max()

# Splitting the dataset
training_validating = pd.DataFrame()
testing_data = pd.DataFrame()

for location, latest_year in latest_year_per_location.items():
    # Training set: Exclude the latest year for each location
    training_validating = pd.concat([training_validating, df_final_sub[(df_final_sub['Location'] == location) & (df_final_sub['Year'] < latest_year - 2)]])
    
    # Testing set: Include only the latest year for each location
    testing_data = pd.concat([testing_data, df_final_sub[(df_final_sub['Location'] == location) & (df_final_sub['Year'].isin([latest_year, latest_year - 1, latest_year - 2]))]])


In [99]:
#since this is imbalance dataset (more non pulse events compared to pulse events, 
#so randomly remove 70% of non pulse events)
zero_rows = training_validating[training_validating['pulse_cluster'] == 0]

#Randomly sample 70% of these zero rows
retained_zero_rows = zero_rows.sample(frac=0.3, random_state=42) 

#Select all rows where 'pulse_cluster' is not 0
non_zero_rows = training_validating[training_validating['pulse_cluster'] != 0]

#Concatenate the retained zero rows with non-zero rows
training_validating = pd.concat([retained_zero_rows, non_zero_rows])

In [100]:
#Prepare training, validating, and testing set
import random
random.seed(710)
X_train_valid = training_validating.drop(['Location', 'pulse_cluster', 'Year', 'Time'], axis = 1).to_numpy()
y_train_valid = training_validating['pulse_cluster'].to_numpy()

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, stratify = y_train_valid)

In [101]:
#Split the data and run on validation test
start_time = time.time()

# Calculate class weights, many ways to do this, one is using sklearn's compute_class_weight
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
weights = {i: class_weights[i] for i in range(len(class_weights))}

# Initialize RandomForestClassifier with class_weight parameter
classifier_full = RandomForestClassifier(random_state=100, class_weight=weights)
classifier_full.fit(X_train, y_train)

y_pred = classifier_full.predict(X_valid)
y_pred_score = classifier_full.predict_proba(X_valid)[:, 1]

end_time = time.time()
print(f"Function ran for {end_time - start_time} seconds.")

print("Performance on validation set: ")
print("Accuracy :", accuracy_score(y_true=y_valid, y_pred=y_pred))
print("Precision:", precision_score(y_true=y_valid, y_pred=y_pred))
print("Recall   :", recall_score(y_true=y_valid, y_pred=y_pred))
print("F1       :", f1_score(y_true=y_valid, y_pred=y_pred))
print("AUROC    :", roc_auc_score(y_true=y_valid, y_score=y_pred_score))

Function ran for 222.0983591079712 seconds.
Performance on validation set: 
Accuracy : 0.9995278062276373
Precision: 0.9988935211712868
Recall   : 0.9997033814514534
F1       : 0.9992982872278392
AUROC    : 0.9999985597886198


In [103]:
#model performance on testing set at each site, not training or validating
#predict on testing set
location_list = []
acc_list = []
pre_list = []
recall_list = []
F1_list = []

df_all = []

for i in df_final_sub['Location'].unique():
    data_sub = testing_data[testing_data['Location'] == i]
    X_test = data_sub.drop(['Location', 'pulse_cluster', 'Year', 'Time'], axis = 1).to_numpy() 
    y_test = data_sub["pulse_cluster"].to_numpy()
    
    y_test_pred = classifier_full.predict(X_test)
    data_sub['pulse_pred'] = y_test_pred
    df_all.append(data_sub)
    y_test_pred_score = classifier_full.predict_proba(X_test)[:, 1]

    location_list.append(i)
    
    acc_list.append(accuracy_score(y_true=y_test, y_pred=y_test_pred))
    pre_list.append(precision_score(y_true=y_test, y_pred=y_test_pred))
    recall_list.append(recall_score(y_true=y_test, y_pred=y_test_pred))
    F1_list.append(f1_score(y_true=y_test, y_pred=y_test_pred))


/var/folders/x2/jr4h0w9926bbmtl5ysq4v78h0000gs/T/ipykernel_98331/414499181.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sub['pulse_pred'] = y_test_pred
/var/folders/x2/jr4h0w9926bbmtl5ysq4v78h0000gs/T/ipykernel_98331/414499181.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sub['pulse_pred'] = y_test_pred
/var/folders/x2/jr4h0w9926bbmtl5ysq4v78h0000gs/T/ipykernel_98331/414499181.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [104]:
#Print out precision and recall performance
df_stat = pd.DataFrame({ 'Location': location_list, 'Accuracy': acc_list, 'Precision': pre_list, 'Recall': recall_list, 'F1': F1_list})
df_stat.sort_values(by='F1')

,Location,Accuracy,Precision,Recall,F1
10,US-Jo2,0.926569,0.268218,0.200733,0.229620
15,US-Rwf,0.899554,0.223278,0.275660,0.246719
5,ES-Agu,0.901541,0.292377,0.227178,0.255686
1,AU-Cpr,0.874548,0.507177,0.189737,0.276162
26,US-Wjs,0.897685,0.649337,0.198442,0.303985
11,US-LS2,0.921050,0.751579,0.194444,0.308957
9,US-Jo1,0.939101,0.861167,0.209598,0.337141
14,US-Rms,0.914412,0.482196,0.271512,0.347408
20,US-SRS,0.948835,0.400802,0.341880,0.369004
12,US-Mpj,0.832290,0.767553,0.255806,0.383726


In [106]:
#load the result, assuming you have saved the RF joblib
classifier_full = load("/Users/ngocnguyen/Library/CloudStorage/Box-Box/Projects/Birch_effect_paper/Results/Datasets/Random_forest/random_forest_model_Sept14_after_EF.joblib")

#applying the trained model on unseen testing set
#predict on testing set
location_list = []

df_all = []

for i in df_final_sub['Location'].unique():
    data_sub = testing_data[testing_data['Location'] == i]
    X_test = data_sub.drop(['Location', 'pulse_cluster', 'Year', 'Time'], axis = 1).to_numpy() 
    y_test = data_sub["pulse_cluster"].to_numpy()
    
    y_test_pred = classifier_full.predict(X_test)
    
    data_sub['pulse_pred'] = y_test_pred
    df_all.append(data_sub)
    location_list.append(i)

/var/folders/x2/jr4h0w9926bbmtl5ysq4v78h0000gs/T/ipykernel_98331/1509831644.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sub['pulse_pred'] = y_test_pred
/var/folders/x2/jr4h0w9926bbmtl5ysq4v78h0000gs/T/ipykernel_98331/1509831644.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sub['pulse_pred'] = y_test_pred
/var/folders/x2/jr4h0w9926bbmtl5ysq4v78h0000gs/T/ipykernel_98331/1509831644.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [107]:
#remove bad predicted pulses (such as short pulses + overlapped pulses)
ML_detect = []
for i in range(28):
    df_sub = df_all[i].reset_index().drop(columns='index')
    df = create_sop_eop_df(df_sub.groupby('Time').apply(remove_bad_pulse))
    df['SITE_ID'] = df_sub['Location'].unique()[0]
    ML_detect.append(df)
    
ML_detect_merge = pd.concat(ML_detect, ignore_index=True)

/var/folders/x2/jr4h0w9926bbmtl5ysq4v78h0000gs/T/ipykernel_98331/43312549.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = create_sop_eop_df(df_sub.groupby('Time').apply(remove_bad_pulse))
/var/folders/x2/jr4h0w9926bbmtl5ysq4v78h0000gs/T/ipykernel_98331/43312549.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = create_sop_eop_df(df_sub.groupby('Time').apply(remove_bad_pulse))
/var/folders/x2/

In [108]:
#Save the predicted Start of Pulse (SOP)
ML_detect_merge.to_csv('/Users/ngocnguyen/Library/CloudStorage/Box-Box/Projects/Birch_effect_paper/Results/Datasets/Random_forest/non-training-sites/ML-pulse-decay-info/ML_ori_NEE_SOP_Sept_14_filter_after_EF.csv', index=False)

In [ ]:
#feature importance
feature_importances = classifier_full.feature_importances_
feature_names = training_validating.drop(['Location', 'pulse_cluster', 'Year', 'Time'], axis = 1).columns

importance_scores = zip(feature_names, feature_importances)
sorted_features = sorted(importance_scores, key=lambda x: x[1], reverse=True)

for feature, importance in sorted_features:
    print(f"{feature}: {importance}")
pd.DataFrame(sorted_features).to_csv('/Users/ngocnguyen/Library/CloudStorage/Box-Box/Projects/Birch_effect_paper/Results/Datasets/Random_forest/28_sites_Feature_Impo.csv', index=False)